In [ ]:
#Necessary imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets,transforms
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

from functools import partial
import tensorflow as tf

import tensorflow as tf
from tensorflow.python.ops import nn
from tensorflow.python.keras import activations, regularizers, initializers, constraints, engine
from tensorflow.python.keras.utils import conv_utils
from tensorflow.python.keras.layers import Layer, deserialize, Conv1D
from tensorflow.python.keras import backend as K
from tensorflow.python.ops import array_ops
from tensorflow.python.keras.models import Sequential,Model
from tensorflow.python.keras.layers import Flatten, Dense,AveragePooling1D,GRU,Convolution1D, MaxPooling1D, AveragePooling1D,Embedding,Input, Dense, Dropout, Flatten

In [ ]:

# The Capsule Architecture
gru_len = 128
Routings = 5
Num_capsule = 10
Dim_capsule = 16
dropout_p = 0.3
rate_drop_dense = 0.3

def squash(x, axis=-1):
    s_squared_norm = K.sum(K.square(x), axis, keepdims=True) + K.epsilon()
    scale = K.sqrt(s_squared_norm) / (0.5 + s_squared_norm)
    return scale * x


# define our own softmax function instead of K.softmax
# because K.softmax can not specify axis.
def softmax(x, axis=-1):
    ex = K.exp(x - K.max(x, axis=axis, keepdims=True))
    return ex / K.sum(ex, axis=axis, keepdims=True)


# define the margin loss like hinge loss
def margin_loss(y_true, y_pred):
    lamb, margin = 0.5, 0.1
    return K.sum(y_true * K.square(K.relu(1 - margin - y_pred)) + lamb * (
        1 - y_true) * K.square(K.relu(y_pred - margin)), axis=-1)


class Capsule(Layer):

    """A Capsule Implement with Pure Keras
    There are two vesions of Capsule.
    One is like dense layer (for the fixed-shape input),
    and the other is like timedistributed dense (for various length input).

    The input shape of Capsule must be (batch_size,
                                        input_num_capsule,
                                        input_dim_capsule
                                       )
    and the output shape is (batch_size,
                             num_capsule,
                             dim_capsule
                            )

    Capsule Implement is from https://github.com/bojone/Capsule/
    Capsule Paper: https://arxiv.org/abs/1710.09829
    """

    def __init__(self,
                 num_capsule,
                 dim_capsule,
                 routings=3,
                 share_weights=True,
                 activation='squash',
                 **kwargs):
        super(Capsule, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings
        self.share_weights = share_weights
        if activation == 'squash':
            self.activation = squash
        else:
            self.activation = activations.get(activation)

    def build(self, input_shape):
        input_dim_capsule = input_shape[-1]
        if self.share_weights:
            self.kernel = self.add_weight(
                name='capsule_kernel',
                shape=(1, input_dim_capsule,
                       self.num_capsule * self.dim_capsule),
                initializer='glorot_uniform',
                trainable=True)
        else:
            input_num_capsule = input_shape[-2]
            self.kernel = self.add_weight(
                name='capsule_kernel',
                shape=(input_num_capsule, input_dim_capsule,
                       self.num_capsule * self.dim_capsule),
                initializer='glorot_uniform',
                trainable=True)

    def call(self, inputs):
        """Following the routing algorithm from Hinton's paper,
        but replace b = b + <u,v> with b = <u,v>.

        This change can improve the feature representation of Capsule.

        However, you can replace
            b = K.batch_dot(outputs, hat_inputs, [2, 3])
        with
            b += K.batch_dot(outputs, hat_inputs, [2, 3])
        to realize a standard routing.
        """

        if self.share_weights:
            hat_inputs = K.conv1d(inputs, self.kernel)
        else:
            hat_inputs = K.local_conv1d(inputs, self.kernel, [1], [1])

        batch_size = K.shape(inputs)[0]
        input_num_capsule = K.shape(inputs)[1]
        hat_inputs = K.reshape(hat_inputs,
                               (batch_size, input_num_capsule,
                                self.num_capsule, self.dim_capsule))
        hat_inputs = K.permute_dimensions(hat_inputs, (0, 2, 1, 3))

        b = K.zeros_like(hat_inputs[:, :, :, 0])
        for i in range(self.routings):
            c = softmax(b, 1)
            o = self.activation(K.batch_dot(c, hat_inputs, [2, 2]))
            if i < self.routings - 1:
                b = K.batch_dot(o, hat_inputs, [2, 3])
                if K.backend() == 'theano':
                    o = K.sum(o, axis=1)

        return o

    def compute_output_shape(self, input_shape):
        return (None, self.num_capsule, self.dim_capsule)


In [ ]:
k=0
X_train=[]
y_train=[]
n=50 #set n equal number of authors present in the dataset
p=50 #set p equal to number of tweets per user you want in training set
z=100 #Z is the total number of tweets per user
for i in range(0,n):  
  for j in range(k,k+p):
    X_train.append(dataset.iloc[j,0])
    y_train.append(dataset.iloc[j,1])
  k+=z

In [ ]:
dataset

,0,1
0,b'India and Benin should broaden trade ties: T...,61771813
1,b'Parliament should have 100 sittings a year: ...,61771813
2,"b""PM launches Cairn India's oil fields in Barm...",61771813
3,b'Jharkhand governor to hold additional charge...,61771813
4,b'British national found dead in Goa village -...,61771813
...,...,...
4995,"b""@SelenaSly2 ooc: *hugs* I'm Great ! Merry Ch...",63611401
4996,b'@SlytherinReject thanks *smiles slightly* an...,63611401
4997,b'@jadeeslytherin It was good ! good !\n',63611401
4998,"b""@aceravenclaw *laughs and starts to calm dow...",63611401


In [ ]:
df = pd.DataFrame({'Text':X_train,'Author':y_train})
df.to_csv('50_tweets_per_user.csv', index=False, encoding='utf-8')

In [ ]:
 
dataset=pd.read_csv('/content/100_tweets_per_user.csv')  #the location of the dataset that need to be processed

k=0
X_train=[]
y_train=[]
n=50 #set n equal number of authors present in the dataset
p=50 #set p equal to number of tweets per user you want in training set
z=100 #Z is the total number of tweets per user
for i in range(0,n):  
  for j in range(k,k+p):
    X_train.append(dataset.iloc[j,0])
    y_train.append(dataset.iloc[j,1])
  k+=z

a=10 #set a equal to number of tweets per user you want in testing set
k=p
X_test=[]
y_test=[]
for i in range(0,n):
  for j in range(k,k+a):
    X_test.append(dataset.iloc[j,0])
    y_test.append(dataset.iloc[j,1])
  k+=z

X_train=pd.DataFrame(X_train)
X_test=pd.DataFrame(X_test)
y_train=pd.DataFrame(y_train)
y_test=pd.DataFrame(y_test)
X_train=X_train.iloc[:,:].values
X_test=X_test.iloc[:,:].values
y_train=y_train.iloc[:,:].values
y_test=y_test.iloc[:,:].values

train=np.concatenate((X_train,y_train),axis=1)
test=np.concatenate((X_test,y_test),axis=1)

np.random.shuffle(train)
np.random.shuffle(test)

train=pd.DataFrame(train)
test=pd.DataFrame(test)

X_train=train.iloc[:,0]
X_test=test.iloc[:,0]
y_train=train.iloc[:,1]
y_test=test.iloc[:,1]


In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
encoder = LabelEncoder()
encoder.fit(y_train)
encoded_Y = encoder.transform(y_train)
# convert integers to dummy variables (i.e. one hot encoded)
y_train = np_utils.to_categorical(encoded_Y)

from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
encoder = LabelEncoder()
encoder.fit(y_test)
encoded_Y = encoder.transform(y_test)
# convert integers to dummy variables (i.e. one hot encoded)
y_test = np_utils.to_categorical(encoded_Y)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X_train)
vectorizer.transform(X_train)
vectorizer.transform(X_test)
tfidf_vector_X = vectorizer.transform(X_train).toarray() 
tfidf_vector_Y = vectorizer.transform(X_test).toarray() 
#tfidf_vector_X = tfidf_vector_X[:, :, None] 
#tfidf_vector_Y = tfidf_vector_Y[:, :, None]

In [ ]:
tfidf_vector_X.shape[1]

7814

In [ ]:
# Splitting of training set into training set and evalution set
from sklearn.model_selection import train_test_split
X_train, X_eval, y_train, y_eval = train_test_split(tfidf_vector_X,y_train, test_size = 0.25, random_state = 0)

In [ ]:
X_train.shape

(1500, 7814, 1)

In [ ]:
# Early Stoppping
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3) 
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)


In [ ]:
def model2(filter_kernels, dense_outputs, maxlen, vocab_size, nb_filter,cat_output):                                                  # For Character Embedding use this model instead of above model
    d = 300                                                             #Embedding Size
    inputs = Input(shape=(maxlen, vocab_size), name='input', dtype='float32')
    conv = Convolution1D(filters=nb_filter, kernel_size=filter_kernels[0], activation='relu',input_shape=(maxlen, vocab_size))(inputs)

    #conv=KernelConv2D(input_shape=(maxlen ,vocab_size),filters=128,kernel_size=1,kernel_function=PolynomialKernel(p=2, trainable_c=True))(inputs)
    #conv = MaxPooling1D(pool_length=3)(conv)

    conv1 = Convolution1D(filters=nb_filter, kernel_size=filter_kernels[1],
                           activation='relu')(conv)
    #conv1 = MaxPooling1D(pool_length=3)(conv1)

    #conv2 = Convolution1D(nb_filter=nb_filter, filter_length=filter_kernels[2],border_mode='valid', activation='relu')(conv1)
    #conv5 = MaxPooling1D(pool_length=3)(conv2)
    conv6= Capsule(num_capsule=1 ,dim_capsule=72, routings=1,share_weights=True)(conv1)
    conv = Flatten()(conv6)

    #Two dense layers with dropout of .5
    #z = Dropout(0.5)(Dense(dense_outputs, activation='relu')(conv))
    # z = Dropout(0.5)(Dense(dense_outputs, activation='relu')(z))

    pred = Dense(cat_output, activation='softmax', name='output')(conv)

    model = Model(inputs=inputs, outputs=pred)

    #sgd = SGD(lr=0.001, momentum=0.9)
    #model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

    return model

In [ ]:
nb_filter = 500
dense_outputs = 256
filter_kernels = [3,4,5]
cat_output = 50 #Set cat_output equal to number of authors present in the dataset
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
model = model2(filter_kernels, dense_outputs,7814, 1, nb_filter, cat_output)
model.summary()


Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 7814, 1)]         0         
_________________________________________________________________
conv1d (Conv1D)              (None, 7812, 500)         2000      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 7809, 500)         1000500   
_________________________________________________________________
capsule (Capsule)            (None, 1, 1, 72)          36000     
_________________________________________________________________
flatten (Flatten)            (None, 72)                0         
_________________________________________________________________
output (Dense)               (None, 50)                3650      
Total params: 1,042,150
Trainable params: 1,042,150
Non-trainable params: 0
____________________________________________

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

model.fit(np.array(X_train), np.array(y_train),validation_data=(np.array(X_eval), np.array(y_eval)),batch_size=32,epochs=100, verbose=1,callbacks=[callback])

Epoch 1/100
 6/47 [==>...........................] - ETA: 15:29 - loss: 3.9157 - accuracy: 0.0156

In [ ]:
# Prediction
y_pred = model.predict(np.array(tfidf_vector_Y))

y_pred=pd.DataFrame(y_pred)
y_pred=y_pred.eq(y_pred.where(y_pred != 0).max(1), axis=0).astype(int)
y_pred=y_pred.iloc[:,:].values

y_test=pd.DataFrame(y_test)
y_test=y_test.eq(y_test.where(y_test != 0).max(1), axis=0).astype(int)
y_test=y_test.iloc[:,:].values

result=[]
for i in range(0,len(y_test)):
  for j in range(0,len(y_test[0])):
    if(y_test[i][j]==1):
      result.append(j)

predicted=[]
for i in range(0,len(y_pred)):
  for j in range(0,len(y_pred[0])):
    if(y_pred[i][j]==1):
      predicted.append(j)

print(result)
print(predicted)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(result,predicted)

from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
print('Confusion Matrix :')
print(cm) 
print('Accuracy Score :',accuracy_score(result, predicted)) 
print('Report : ')
print(classification_report(result, predicted))